In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installing necessary applications for 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/"
!sudo apt-get update
!sudo apt-get -y install cuda

# Install JoeyNMT
! git clone https://github.com/may-/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.9.0
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

--2021-09-22 05:04:43--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2021-09-22 05:04:43 (12.6 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.5t3jpepyCE/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Er

In [4]:
!pip install sacremoses

## clone indic nlp resources repo
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
## install indic nlp library
!pip install indic-nlp-library
## export indic nlp resource library
import os
os.environ["INDIC_RESOURCES_PATH"] = "/content/indic_nlp_resources"

     |████████████████████████████████| 895 kB 9.4 MB/s 
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 34.70 MiB/s, done.
Resolving deltas: 100% (53/53), done.
     |████████████████████████████████| 40 kB 5.1 MB/s 
     |████████████████████████████████| 2.8 MB 10.8 MB/s 


In [5]:
def create_model_config(name, src_lang, tgt_lang):
  model_type = "bpe"
  use_cuda = "true"
  config = """
  name: "{name}"

  data:
      src: "{src_lang}"
      trg: "{tgt_lang}"
      train: "data/{name}/train.{model_type}"
      dev:   "data/{name}/dev.{model_type}"
      test:   "data/{name}/test.{model_type}"
      level: "{model_type}"
      lowercase: False
      max_sent_length: 60
      src_vocab: "data/{name}/vocab.txt"
      trg_vocab: "data/{name}/vocab.txt"

  testing:
      beam_size: 5
      alpha: 1.0

  training:
      load_model: "models/{name}/best.ckpt"
      random_seed: 42
      optimizer: "adam"
      normalization: "tokens"
      adam_betas: [0.9, 0.999] 
      scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
      patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
      learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
      learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
      decrease_factor: 0.7
      loss: "crossentropy"
      learning_rate: 0.0003
      learning_rate_min: 0.00000001
      weight_decay: 0.0
      label_smoothing: 0.1
      batch_size: 4096
      batch_type: "token"
      eval_batch_size: 3600
      eval_batch_type: "token"
      batch_multiplier: 1
      early_stopping_metric: "ppl"
      epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
      validation_freq: 1000          # TODO: Set to at least once per epoch.
      logging_freq: 100
      eval_metric: "bleu"
      model_dir: "models/{name}"
      overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
      shuffle: True
      use_cuda: {use_cuda}
      max_output_length: 130
      print_valid_sents: [0, 1, 2, 3]
      keep_best_ckpts: 3

  model:
      initializer: "xavier"
      bias_initializer: "zeros"
      init_gain: 1.0
      embed_initializer: "xavier"
      embed_init_gain: 1.0
      tied_embeddings: True
      tied_softmax: True
      encoder:
          type: "transformer"
          num_layers: 6
          num_heads: 4             # TODO: Increase to 8 for larger data.
          embeddings:
              embedding_dim: 256   # TODO: Increase to 512 for larger data.
              scale: True
              dropout: 0.2
          # typically ff_size = 4 x hidden_size
          hidden_size: 256         # TODO: Increase to 512 for larger data.
          ff_size: 1024            # TODO: Increase to 2048 for larger data.
          dropout: 0.3
      decoder:
          type: "transformer"
          num_layers: 6
          num_heads: 4              # TODO: Increase to 8 for larger data.
          embeddings:
              embedding_dim: 256    # TODO: Increase to 512 for larger data.
              scale: True
              dropout: 0.2
          # typically ff_size = 4 x hidden_size
          hidden_size: 256         # TODO: Increase to 512 for larger data.
          ff_size: 1024            # TODO: Increase to 2048 for larger data.
          dropout: 0.3
  """.format(name = name, src_lang = src_lang, tgt_lang = tgt_lang, model_type = model_type, use_cuda = use_cuda)
  with open("joeynmt/configs/{name}.yaml".format(name = name), 'w') as f:
      f.write(config)

In [6]:
def setup_enhi_models():
  import os

  joey_nmt_home = "/content/joeynmt"
  gdrive_project_home = "/content/drive/MyDrive/DA"
  os.environ["joey_nmt_home"] = joey_nmt_home
  os.environ["gdrive_project_home"] = gdrive_project_home


  models = ['pm_india_nhs_inform_bt_baseline', 'pm_india_nhs_inform_bt_da1', 'pm_india_nhs_inform_bt_da2']

  for name in models:
      os.environ["name"] = name

      if name == "pm_india_nhs_inform_bt_baseline":
        model_data_dir = "backtranslationhi_en_pmi_nhs_backtranslate_5000/model_data"
        model_bin_dir = "model_pmi_nhs_backtranslate_100epochs_new/hien_transformer_pmi_nhs_backtranslate"
      elif name == "pm_india_nhs_inform_bt_da1":
        model_data_dir = "hi_en_pmi_nhs_da_bt_5000/"
        model_bin_dir = "model_pmi_nhs_da_bt_30epochs/hien_transformer_pmi_nhs_da_bt"
      elif name == "pm_india_nhs_inform_bt_da2":
        model_data_dir = "hi_en_pmi_nhs_da_bt_retrain_5000/"
        model_bin_dir = "model_pmi_nhs_da_bt_retrain_30epochs/hien_transformer_pmi_nhs_da_bt_retrain"

      os.environ["model_data_dir"] = model_data_dir
      os.environ["model_bin_dir"] = model_bin_dir
      
      ### set up data for model
      print("Clean up data and models directory in Joey NMT filesystem")
      !rm -rf $joey_nmt_home/data/$name/*
      !rm -rf $joey_nmt_home/models/$name/*
      print("Setting up the model:", name)
      print("Creating relevant directories in Joey NMT filesystem")
      !mkdir -p $joey_nmt_home/data/$name
      !mkdir -p $joey_nmt_home/models/$name
      print("Copying model's data directory from GDRIVE: ")
      !cp -R $gdrive_project_home/$model_data_dir/* $joey_nmt_home/data/$name
      print("Copying model's bin directory from GDRIVE: ")
      !cp -R $gdrive_project_home/$model_bin_dir/* $joey_nmt_home/models/$name
      print()


In [7]:
setup_enhi_models()

Clean up data and models directory in Joey NMT filesystem
Setting up the model: pm_india_nhs_inform_bt_baseline
Creating relevant directories in Joey NMT filesystem
Copying model's data directory from GDRIVE: 
Copying model's bin directory from GDRIVE: 

Clean up data and models directory in Joey NMT filesystem
Setting up the model: pm_india_nhs_inform_bt_da1
Creating relevant directories in Joey NMT filesystem
Copying model's data directory from GDRIVE: 
Copying model's bin directory from GDRIVE: 

Clean up data and models directory in Joey NMT filesystem
Setting up the model: pm_india_nhs_inform_bt_da2
Creating relevant directories in Joey NMT filesystem
Copying model's data directory from GDRIVE: 
Copying model's bin directory from GDRIVE: 



In [13]:
!whoami

root


In [15]:
!cp /content/joeynmt/data/pm_india_nhs_inform_bt_baseline/vocab.txt /content/joeynmt/data/pm_india_nhs_inform_bt_da1
!cp /content/joeynmt/data/pm_india_nhs_inform_bt_baseline/vocab.txt /content/joeynmt/data/pm_india_nhs_inform_bt_da2

In [16]:
create_model_config("pm_india_nhs_inform_bt_baseline", "hi", "en")
create_model_config("pm_india_nhs_inform_bt_da1", "hi", "en")
create_model_config("pm_india_nhs_inform_bt_da2", "hi", "en")

In [17]:
!cd joeynmt; python3 -m joeynmt test /content/joeynmt/configs/pm_india_nhs_inform_bt_da1.yaml --output_path pm_india_nhs_inform_bt_da1

2021-09-22 05:21:30,066 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-22 05:21:30,066 - INFO - joeynmt.data - Building vocabulary...
2021-09-22 05:21:30,501 - INFO - joeynmt.data - Loading dev data...
2021-09-22 05:21:30,508 - INFO - joeynmt.data - Loading test data...
2021-09-22 05:21:30,514 - INFO - joeynmt.data - Data loaded.
2021-09-22 05:21:30,576 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-09-22 05:21:30,577 - INFO - joeynmt.prediction - Loading model from models/pm_india_nhs_inform_bt_da1/87100.ckpt
2021-09-22 05:21:34,526 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-22 05:21:34,721 - INFO - joeynmt.model - Enc-dec model built.
2021-09-22 05:21:34,796 - INFO - joeynmt.prediction - Decoding on dev set (data/pm_india_nhs_inform_bt_da1/dev.bpe.en)...
2021-09-22 05:22:01,498 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-09-22 05:22:01,498 - WARNING - sacr

In [ ]:
!cd joeynmt; python3 -m joeynmt test /content/joeynmt/configs/pm_india_nhs_inform_da2.yaml --output_path pm_india_nhs_inform_da2_translations.txt

2021-09-21 23:51:39,966 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-21 23:51:39,966 - INFO - joeynmt.data - Building vocabulary...
2021-09-21 23:51:40,414 - INFO - joeynmt.data - Loading dev data...
2021-09-21 23:51:40,421 - INFO - joeynmt.data - Loading test data...
2021-09-21 23:51:40,427 - INFO - joeynmt.data - Data loaded.
2021-09-21 23:51:40,445 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-09-21 23:51:40,445 - INFO - joeynmt.prediction - Loading model from models/pm_india_nhs_inform_da2/latest.ckpt
2021-09-21 23:51:43,475 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-21 23:51:43,690 - INFO - joeynmt.model - Enc-dec model built.
2021-09-21 23:51:43,758 - INFO - joeynmt.prediction - Decoding on dev set (data/pm_india_nhs_inform_da2/dev.bpe.hi)...
2021-09-21 23:52:00,159 - INFO - joeynmt.prediction -  dev bleu[13a]:  18.84 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-09-21 2